# Retrograde Analyse für das Schach-Endspiel
_Beschreibung_

Bekannte Probleme:
* Nicht optimiert für ein Spielbrett mit Bauern

Mögliche Verbesserungen:
* Das Initialisieren des Notebooks in Funktionen kapseln, um erneute Ausführungen einfacher zu gestalten.

## Imports
Für das Notebook benötigte Imports
* chess: Die Python-Schach-Bibliothek, mit welcher Schachbretter dargestellt werden, Züge und Zustände ausgewertet etc.
* Factorial aus der Math-Library: Die Fakultät (n!) wird zum Aufstellen der Board_List benötigt
* Clear_output aus der IPython-display-Library: Eine Funktion, die für die Fortschrittsanzeige von Berechnungen verwendet wird.

In [34]:
import chess
from math import factorial
from IPython.display import clear_output
import json


## Eingabe des Schachbretts, mit der Ausgangssituation
Langfristig wird dies durch [@NicoMiller's](https://github.com/millerni) Board-Setup-Code ersetzt.
Aktuell wird eine Fen eingegeben.
Mit der Forsyth-Edwards-Notation kann der Zustand eines Schachbretts kompakt notiert werden. Mehr Infos zum Thema Fen finden sich [hier](https://www.chess.com/terms/fen-chess).

Das gegenwärtige Beispiel `8/8/8/8/4k3/8/8/K6R w - - 0 1` beschreibt das Szenario König und Turm gegen König.

Weiter wird in der folgenden Zelle das erste Board-Objekt erstellt.


In [35]:
fen = "8/8/8/8/4k3/8/8/K6R w - - 0 1"
#fen = "8/8/8/8/4k3/8/2P5/K7 w - - 0 1"
board = chess.Board(fen)

## Erstellen der *pieceList*
Die _Figurenliste_ enthält alle Figuren, welche sich auf dem Brett befinden in beliebiger Reihenfolge.

Das Erstellen dieser Liste erfolgt in einem mehrschritten Prozess:

1. Ein Dictionary mit Figuren und Positionen von der Chess-Library abfragen.
   Beispiel-Eintrag: `{28: Piece.from_symbol('k')}`
   Hierbei ist die 28 ein Quadrat auf dem Brett und k ein König.
2. Über das Dictionary iterieren und die Figuren einer Liste hinzufügen.

Beispielhafter Auszug aus dem Ergebnis:
`[Piece.from_symbol('k')]`

In [36]:
#get the piece map from the library
tmp_piece_map = board.piece_map()
pieceList = []
#iterate over the map and append the pieces to the list
for location in tmp_piece_map:
    pieceList.append(tmp_piece_map[location])

## *board_list* erstellen
Die Brettliste ist ein Python-Listen-Objekt, welches `64! / (64-count(pieces))!` leere Schachbretter enthält.
In einem weiteren Schritt kann auf dem ersten Quadrat die erste Figur platziert werden, die zweite Figur auf den Quadraten zwei bis 64, die dritte auf den verbleibenden 62 Quadraten etc..
Somit gibt es (bei insgesamt drei Figuren) `63*62 = 3.906` verschiedene Bretter, mit _Figur 1_ auf dem ersten Quadrat und insgesamt `64*63*62 = 249.984` verschiedene Bretter.

Dieser Zustand wird erreicht, indem ein neues Board-Objekt erstellt und alle Figuren entfernt werden.
Dieses Objekt wird dann `64! / (64-count(pieces))!` Mal einer Liste hinzugefügt.

In [37]:
tmp_board = chess.Board()
tmp_board.clear()
board_list = []
list_length = factorial(64) // factorial(64 - len(pieceList))
if chess.Piece.from_symbol('P') in pieceList:
    list_length = list_length * 2

for i in range(list_length):
    board_list.append(tmp_board.copy())

## Die rekursive placePieces-Hilfsfunktion

Funktions-Argumente:
* boardList: Eine Liste, mit leeren oder teilweise mit Figuren gefüllten Board-Objekten
* pieceList: Eine Liste, welche alle Figuren, welche auf dem Board platziert werden sollen, enthält.
* pieceIndexStart: Index der Figur, welche platziert werden soll
* start: Index der ersten Position in *boardList*, an welcher Figuren platziert werden sollen.
* stop: Index+1 der letzten Position in *boardList*, an welcher Figuren platziert werden sollen. (`[start,strop)`)

Ergebnis der Ausführung:
Die Funktion platziert rekursiv Figuren aus der _pieceList_ mit einem Index >= _pieceIndexStart_
auf allen Brettern in der _boardList_ welche im Index-Bereich _start_ bis _stop - 1_ liegen.

Nebeneffekte:
Die Funktion verändert die als argument übergebene Board-Objekt-Liste _boardList_.

Algorithmus:
* Den Versatz für die aktuelle Figur berechnen.
  Der Versatz beschreibt, auf wie vielen Boards die aktuelle Figur im selben Quadrat platziert werden muss.
  Beispiel: Befinden sich zwei Figuren in der _FigurenListe_, dann muss die erste Figur auf allen 64 Quadraten platziert werden.
  Die zweite Figur muss, für jede der 64 Positionen von _Figur 1_ auf den verbleibenden 63 Quadraten platziert werden.
  ```
  O|     O|X  O|  O|
  --- -> ---  --- ---
   |      |   X|   |X

   |O    X|O   |O  |O
  --- -> ---  --- ---
   |      |   X|   |X
  ```
  Auf diesem 2x2 Spielbrett hat die zweite Figur drei mögliche Positionen, nachdem die erste Figur platziert wurde.
  Bei jedem vierten Brett muss die erste Figur also auf einem anderem Quadrat platziert werden.
  Dieser Schritt kann auf ein 64x64 Schachbrett skaliert werden.
  In diesem simplen Beispiel wäre der Versatz drei.
  Für größere Spielbretter, wie ein Schachbrett kann der Versatz mit der folgenden Formel berechnet werden.
  `offset = (stop - start) // (64 - (len(pieceList) - 1) + pieceIndexStart)`
* Über die _BrettListe_ iterieren, beginnend beim Index _start_
    * Die Figur, welche sich in der _pieceList_ an Index _pieceIndexStart_ befindet auswählen.
    * Die Figur auf dem ersten leeren Spielfeld platzieren. (Wiederholungen dieses Schrittes zählen)
    * Wenn der Versatz erreicht wird:
        * Wenn weitere Figuren platziert werden müssen (`len(pieceList) > pieceIndexStart`), die Funktion rekursiv mit
          höherem _pieceIndexStart_ und neuem _start_ und _stop_ Werten aufrufen.
          Der neue Wert für _stop_ ist der aktuelle Board-Index - 1.
          Der neue Wert für _start_ ist der neue _stop_ Wert weniger des Versatzes.
        * Das nächste Quadrat auswählen (Index erhöhen).

Hinweis:
In diesem Schritt wird __nicht__ überprüft, ob die erzeugten Boards Valide sind.
Dies muss wird in einem nächsten Schritt getan werden.



In [38]:
#TODO: Comments
def placePieces(boardList, pieceList, pieceIndexStart, start, stop):
    squareIndex = -1 #first Run increments to 0
    offset = (stop - start) // (64 - (len(pieceList) - 1) + pieceIndexStart)
    for i in range(start, stop):
        if i % offset == 0:
            if pieceIndexStart < (len(pieceList)-1) and i - offset > 0:
                placePieces(boardList, pieceList, pieceIndexStart + 1, i - offset, i)
            squareIndex = squareIndex + 1
        square = chess.Square(squareIndex)
        if boardList[i].piece_at(square) is None:
            boardList[i].set_piece_at(square, pieceList[pieceIndexStart])

## Alle möglichen Kombinationen der Figuren auf dem Schachbrett erzeugen
Hierfür wird die _placePieces_ Funktion mit der zuvor erzeugten *board_list* und der _pieceList_ aufgerufen.
Als Index für die Figur wird 0 (erste Figur in der Liste) verwendet.
Als _start_ wird 0 (erstes Schachbrett) verwendet. Als _stop_ wird die Größe der Liste (letztes Brett) verwendet.

In [39]:
if not chess.Piece.from_symbol('P') in pieceList:
    placePieces(board_list, pieceList, 0, 0, len(board_list))
else:
    placePieces(board_list, pieceList, 0, 0, len(board_list) // 2)
    for piece in pieceList:
        if piece == chess.Piece.from_symbol('P'):
            pieceList.remove(piece)
            pieceList.append(chess.Piece.from_symbol('Q'))
            break
    placePieces(board_list, pieceList, 0, len(board_list) // 2, len(board_list))
    pieceList.remove(chess.Piece.from_symbol('Q'))
    pieceList.append(chess.Piece.from_symbol('P'))

## Boards aus Einhaltung der Schachregeln überprüfen und invalide entfernen
Über alle Schachbretter in der Liste iterieren und den Status mittels der Chess-Library überprüfen.
Wenn das Board Valide ist, wird es einer neuen Liste hinzugefügt.
Anschließend wird derselbe Schritt mit dem anderen Spieler am Zug durchgeführt, um wirklich alle Kombinationen abzudecken.
Die neue Liste S enthält alle validen Kombinationen der Figuren auf einem Schachbrett.

In [40]:
S = []
for board in board_list:
    #Board with white having the next turn
    board.turn = chess.WHITE
    if board.is_valid():
        S.append(board)
    boardBlackMove = board.copy()
    boardBlackMove.turn = chess.BLACK
    #The same board with black having the next turn
    if boardBlackMove.is_valid():
        S.append(boardBlackMove)

KeyboardInterrupt: 

## Gewonnene Spielbretter finden und zu S\_0 hinzufügen
* Über den Inhalt von S iterieren.
* Status-Informationen von der Bibliothek abfragen und überprüfen, ob ein Spieler gewonnen hat.
* Wenn ja, das Objekt zu *S\_0* hinzufügen.
  Weiter werden die Objekte in ihrer String-Representation zu *S\_0\_ASCII* hinzugefügt.
* Wenn nicht, das Objekt in *S\_tmp* hinzufügen. (Dies ist effizienter als das Objekt aus der Liste S zu entfernen)
* Die ursprüngliche Liste S mit *S\_tmp* ersetzen.

In [ ]:
S_tmp = []
S_ASCII = []
S_0 = []
S_0_ASCII = []
S_WHITE_ASCII = []

for board in S:
    outcome = board.outcome()
    # If True Game has finished in a certain way
    if outcome is not None:
        # A winner has been determined
        if outcome.winner is not None and board.is_valid():
            S_0.append(board)
            S_0_ASCII.append((board.turn,board.__str__()))
    # Every other board
    else:
        S_tmp.append(board)
        S_ASCII.append((board.turn, board.__str__()))
S = S_tmp



## OneStepAway Funktion
Diese Funktion wählt (aus zwei Listen mit Board-Objekten) die Spielbretter aus der zweiten Liste, welche einen Spielzug von einem
Spielbrett in der ersten Liste entfernt sind.

Funktionsargumente:
* S_n: Die Liste mit Board-Objekten, deren Spielbretter sich n Züge vom Sieg entfernt befinden
* S: Die Liste, aus welcher die nächste Stufe ausgewählt werden soll

Nebeneffekte:
Keine.
Alle als Parameter übergebenen Listen bleiben unverändert.

Algorithmus:
* Für effizientere Vergleiche für jedes Objekt in *S* die ASCII-Representation speichern.
* Über die Objekte in *S_n* iterieren
  * Den Spieler, welcher am Zug ist wechseln (Da um im aktuellen Zustand anzukommen der andere Spieler einen Zug gemacht hat)
  * pseudo legale Spielzüge von der Bibliothek berechnen Lassen
    Ein pseudo legaler Spielzug ist ein Spielzug, welcher die grundsätzlichen Bewegungsregeln der Figur einhält,
    aber das Schachbrett unter Umständen in einem nicht regelkonformen Zustand versetzt.
  * Über diese Spielzüge iterieren
    * Den Zug ausführen
    * Überprüfen, ob das Board sich in einem erlaubten Zustand befindet.
      Wenn nein, den Schleifendurchlauf abbrechen.
    * Mittels der ASCII-Representation überprüfen, ob das modifizierte Spielbrett in *S* gefunden wird.
    * Wenn es gefunden wird, die ASCII-Representation zu *s_n1_ascii* hinzufügen
    * Den Zug rückgängig machen
  * Den Spieler, welcher ursprünglich am Zug war wiederherstellen.
* Über *S* iterieren
  * Wenn sich das Spielbrett in *s_n1_ascii* befindet, wird es *s_n1* hinzugefügt.
  * Wenn sich das Spielbrett nicht in *s_n1_ascii* befindet, wird es *s_tmp* hinzugefügt.
* *s_n1* und *s_tmp* als Tupel zurückgeben.

In [ ]:
def one_step_away(S_n, S, s_white_ascii, iterationCount):
    #variables
    s_n1 = []
    s_n1_tmp = []
    s_n1_ascii = []
    s_ascii = []
    s_n_ascii = []
    s_tmp = []

    #create temporary list for comparison
    for chessBoard in S:
        s_ascii.append((chessBoard.turn, chessBoard.__str__()))

    for chessBoard in S_n:
        s_n_ascii.append((chessBoard.turn, chessBoard.__str__()))

    for i in range(len(S_n)):

        if iterationCount is not None:
            status = "Calculating S" + str(iterationCount) + " - Board " + str(i+1) + " of " + str(len(S_n)) + " from S" + str(iterationCount-1)
        else:
            status = "Board " + str(i+1) + " of " + str(len(S_n))
        clear_output(wait=True)
        print(status)

        chessBoard = S_n[i]
        chessBoard.turn = chessBoard.turn ^ True

        pawn_at = []
        for pawn in chessBoard.pieces(chess.PAWN, True):
            pawn_at.append(chess.square_name(pawn))

        for pLMove in chessBoard.pseudo_legal_moves:
            if chess.square_name(pLMove.from_square) not in pawn_at:
                chessBoard.push(pLMove)
                if not chessBoard.is_valid():
                    chessBoard.pop()
                    continue
                if (chessBoard.turn ^ True, chessBoard.__str__()) in s_ascii:
                    s_n1_ascii.append((chessBoard.turn ^ True, chessBoard.__str__()))
                chessBoard.pop()

        for pawn in chessBoard.pieces(chess.PAWN, True):
            if chessBoard.piece_at(pawn - 9) is None:
                chessBoard.remove_piece_at(pawn)
                chessBoard.set_piece_at(pawn - 9, chess.Piece.from_symbol('P'))
                print(chessBoard)
                if chessBoard.is_valid():
                    if (chessBoard.turn ^ True, chessBoard.__str__()) in s_ascii:
                        s_n1_ascii.append((chessBoard.turn ^ True, chessBoard.__str__()))
                chessBoard.remove_piece_at(pawn - 9)
                chessBoard.set_piece_at(pawn, chess.Piece.from_symbol('P'))

        queen = check_top_row(chessBoard)
        if queen:
            chessBoard.remove_piece_at(queen)
            chessBoard.set_piece_at(queen, chess.Piece.from_symbol('P'))
            if chessBoard.is_valid():
                if (chessBoard.turn ^ True, chessBoard.__str__()) in s_ascii:
                    s_n1_ascii.append((chessBoard.turn ^ True, chessBoard.__str__()))
            chessBoard.remove_piece_at(queen)
            chessBoard.set_piece_at(queen, chess.Piece.from_symbol('Q'))


        chessBoard.turn = chessBoard.turn ^ True

    clear_output(wait=True)
    if iterationCount is not None:
        print("Calculating S" + str(iterationCount) + " - Mapping ASCII boards to board objects")
    else:
        print("Mapping ASCII boards to board objects")
    #create return lists
    for chessBoard in S:
        if(chessBoard.turn, chessBoard.__str__()) in s_n1_ascii:
            s_n1.append(chessBoard)
        else:
            s_tmp.append(chessBoard)


    clear_output(wait=True)
    if iterationCount is not None:
        print("Calculating S" + str(iterationCount) + " - Checking Black Moves for determinism")
    else:
        print("Checking Black Moves for determinism")


    # Only needed for Black-Moves
    for chessBoard in s_n1:
        include = True
        if not chessBoard.turn:
            for move in chessBoard.legal_moves:
                chessBoard.push(move)
                str_rep = (chessBoard.turn, chessBoard.__str__())
                chessBoard.pop()
                if str_rep not in s_white_ascii:
                    include = False
        if include:
            s_n1_tmp.append(chessBoard)
        else:
            s_tmp.append(chessBoard)
    s_n1 = s_n1_tmp

    # Only needed for White-Moves
    if iterationCount % 2 == 1:
        s_white_ascii = s_white_ascii + s_n1_ascii


    clear_output(wait=True)
    print("Done with S" + str(iterationCount) + " - " + str(len(s_tmp)) + " Boards remaining in S")



    return s_n1, s_tmp, s_white_ascii

def check_top_row(board):
    for i in range(56, 64):
        if board.piece_type_at(i) == chess.QUEEN:
            return i

        return False

## Alle Spielbretter finden, welche einen Zug vom Gewinn entfernt sind.

In [ ]:
S_1, S, S_WHITE_ASCII = one_step_away(S_0, S, S_WHITE_ASCII, 1)

## In einer Schleife die Liste mit S_n berechnen
//TODO: Kommentare

In [ ]:
S_n_sequence = []
S_n_sequence.append(S_0)
S_n_sequence.append(S_1)
while True:

    S_n_new, S, S_WHITE_ASCII = one_step_away(S_n_sequence[-1], S, S_WHITE_ASCII, len(S_n_sequence))
    if not S_n_new: #an empty list is false
        break
    S_n_sequence.append(S_n_new)

print("Completed")
print(len(S_n_sequence))
print(len(S))

## Ascii Representationen erstellen

In [ ]:
S_n_seq_ascii_boards_list = []

for i in range(len(S_n_sequence)):
    S_n_ascii_boards = []
    for board in S_n_sequence[i]:
        S_n_ascii_boards.append((board.turn, board.__str__()))
    S_n_seq_ascii_boards_list.append(S_n_ascii_boards)

## Exporting the list

In [ ]:
S_n_seq_ascii = []
S_ascii = []

for i in range(len(S_n_sequence)):
    S_n_ascii = []
    for board in S_n_sequence[i]:
        S_n_ascii.append(board.fen())
    S_n_seq_ascii.append(S_n_ascii)

for board in S:
    S_ascii.append(board.fen())

f = open("S_n_seq.json", "w")
f.write(json.dumps(S_n_seq_ascii))
f.close()

f = open("S_rem.json", "w")
f.write(json.dumps(S_ascii))
f.close()


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c2d65970-ce08-4505-aa56-a8fa174656aa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>